# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f3f9cb23970>
├── 'a' --> tensor([[ 0.7133, -1.2241,  0.4244],
│                   [ 0.7915, -0.3099,  0.8456]])
└── 'x' --> <FastTreeValue 0x7f3f9cb1db50>
    └── 'c' --> tensor([[-0.7565,  1.0084, -1.9662, -0.6963],
                        [ 0.5654,  0.0600, -0.7498, -0.6859],
                        [ 1.1125, -1.1716, -0.7321, -0.3056]])

In [4]:
t.a

tensor([[ 0.7133, -1.2241,  0.4244],
        [ 0.7915, -0.3099,  0.8456]])

In [5]:
%timeit t.a

86.6 ns ± 2.04 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f3f9cb23970>
├── 'a' --> tensor([[-0.0179, -0.9719,  0.8975],
│                   [-1.3354, -0.3030,  0.0835]])
└── 'x' --> <FastTreeValue 0x7f3f9cb1db50>
    └── 'c' --> tensor([[-0.7565,  1.0084, -1.9662, -0.6963],
                        [ 0.5654,  0.0600, -0.7498, -0.6859],
                        [ 1.1125, -1.1716, -0.7321, -0.3056]])

In [7]:
%timeit t.a = new_value

101 ns ± 0.252 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.7133, -1.2241,  0.4244],
               [ 0.7915, -0.3099,  0.8456]]),
    x: Batch(
           c: tensor([[-0.7565,  1.0084, -1.9662, -0.6963],
                      [ 0.5654,  0.0600, -0.7498, -0.6859],
                      [ 1.1125, -1.1716, -0.7321, -0.3056]]),
       ),
)

In [10]:
b.a

tensor([[ 0.7133, -1.2241,  0.4244],
        [ 0.7915, -0.3099,  0.8456]])

In [11]:
%timeit b.a

79.3 ns ± 0.435 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.3783, -0.5066, -1.2840],
               [ 0.2640, -0.6292, -1.2595]]),
    x: Batch(
           c: tensor([[-0.7565,  1.0084, -1.9662, -0.6963],
                      [ 0.5654,  0.0600, -0.7498, -0.6859],
                      [ 1.1125, -1.1716, -0.7321, -0.3056]]),
       ),
)

In [13]:
%timeit b.a = new_value

650 ns ± 0.35 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.05 µs ± 1.86 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.5 µs ± 8.38 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

195 µs ± 23.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

185 µs ± 2.65 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f3ebcb83fd0>
├── 'a' --> tensor([[[ 0.7133, -1.2241,  0.4244],
│                    [ 0.7915, -0.3099,  0.8456]],
│           
│                   [[ 0.7133, -1.2241,  0.4244],
│                    [ 0.7915, -0.3099,  0.8456]],
│           
│                   [[ 0.7133, -1.2241,  0.4244],
│                    [ 0.7915, -0.3099,  0.8456]],
│           
│                   [[ 0.7133, -1.2241,  0.4244],
│                    [ 0.7915, -0.3099,  0.8456]],
│           
│                   [[ 0.7133, -1.2241,  0.4244],
│                    [ 0.7915, -0.3099,  0.8456]],
│           
│                   [[ 0.7133, -1.2241,  0.4244],
│                    [ 0.7915, -0.3099,  0.8456]],
│           
│                   [[ 0.7133, -1.2241,  0.4244],
│                    [ 0.7915, -0.3099,  0.8456]],
│           
│                   [[ 0.7133, -1.2241,  0.4244],
│                    [ 0.7915, -0.3099,  0.8456]]])
└── 'x' --> <FastTreeValue 0x7f3ebcb83fa0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

39.8 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f3f74dc92e0>
├── 'a' --> tensor([[ 0.7133, -1.2241,  0.4244],
│                   [ 0.7915, -0.3099,  0.8456],
│                   [ 0.7133, -1.2241,  0.4244],
│                   [ 0.7915, -0.3099,  0.8456],
│                   [ 0.7133, -1.2241,  0.4244],
│                   [ 0.7915, -0.3099,  0.8456],
│                   [ 0.7133, -1.2241,  0.4244],
│                   [ 0.7915, -0.3099,  0.8456],
│                   [ 0.7133, -1.2241,  0.4244],
│                   [ 0.7915, -0.3099,  0.8456],
│                   [ 0.7133, -1.2241,  0.4244],
│                   [ 0.7915, -0.3099,  0.8456],
│                   [ 0.7133, -1.2241,  0.4244],
│                   [ 0.7915, -0.3099,  0.8456],
│                   [ 0.7133, -1.2241,  0.4244],
│                   [ 0.7915, -0.3099,  0.8456]])
└── 'x' --> <FastTreeValue 0x7f3ebdc5c2e0>
    └── 'c' --> tensor([[-0.7565,  1.0084, -1.9662, -0.6963],
                        [ 0.5654,  0.0600, -0.7498, -0.6859],
                 

In [23]:
%timeit t_cat(trees)

37.4 µs ± 74.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

75.2 µs ± 197 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.7133, -1.2241,  0.4244],
                [ 0.7915, -0.3099,  0.8456]],
       
               [[ 0.7133, -1.2241,  0.4244],
                [ 0.7915, -0.3099,  0.8456]],
       
               [[ 0.7133, -1.2241,  0.4244],
                [ 0.7915, -0.3099,  0.8456]],
       
               [[ 0.7133, -1.2241,  0.4244],
                [ 0.7915, -0.3099,  0.8456]],
       
               [[ 0.7133, -1.2241,  0.4244],
                [ 0.7915, -0.3099,  0.8456]],
       
               [[ 0.7133, -1.2241,  0.4244],
                [ 0.7915, -0.3099,  0.8456]],
       
               [[ 0.7133, -1.2241,  0.4244],
                [ 0.7915, -0.3099,  0.8456]],
       
               [[ 0.7133, -1.2241,  0.4244],
                [ 0.7915, -0.3099,  0.8456]]]),
    x: Batch(
           c: tensor([[[-0.7565,  1.0084, -1.9662, -0.6963],
                       [ 0.5654,  0.0600, -0.7498, -0.6859],
                       [ 1.1125, -1.1716, -0.7321, -0.3056]],
         

In [26]:
%timeit Batch.stack(batches)

99.9 µs ± 232 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.7133, -1.2241,  0.4244],
               [ 0.7915, -0.3099,  0.8456],
               [ 0.7133, -1.2241,  0.4244],
               [ 0.7915, -0.3099,  0.8456],
               [ 0.7133, -1.2241,  0.4244],
               [ 0.7915, -0.3099,  0.8456],
               [ 0.7133, -1.2241,  0.4244],
               [ 0.7915, -0.3099,  0.8456],
               [ 0.7133, -1.2241,  0.4244],
               [ 0.7915, -0.3099,  0.8456],
               [ 0.7133, -1.2241,  0.4244],
               [ 0.7915, -0.3099,  0.8456],
               [ 0.7133, -1.2241,  0.4244],
               [ 0.7915, -0.3099,  0.8456],
               [ 0.7133, -1.2241,  0.4244],
               [ 0.7915, -0.3099,  0.8456]]),
    x: Batch(
           c: tensor([[-0.7565,  1.0084, -1.9662, -0.6963],
                      [ 0.5654,  0.0600, -0.7498, -0.6859],
                      [ 1.1125, -1.1716, -0.7321, -0.3056],
                      [-0.7565,  1.0084, -1.9662, -0.6963],
                      [ 0.5654,  

In [28]:
%timeit Batch.cat(batches)

181 µs ± 328 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

476 µs ± 28.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
